C:\Users\rahmanar\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [ ]:
# #############################################################################
# Importing necessary libraries for creating model

from keras.models import model_from_json
from keras.models import load_model
import os
import pandas as pd
import numpy as np
import keras
#import dlib #Importing library
import cv2 # importing open CV library
import face_recognition # importing face recognition library
import imutils # Package of convenience functions

# #############################################################################
# Reading the trained model from the folder or server
import time
start = time.time()

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

from PIL import Image
from skimage import io
import matplotlib.pyplot as plt


def detect_faces(image):                    # Create a face detector

    face_detector = dlib.get_frontal_face_detector()
    
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames


def resize_im(image, hight, width):          # Resizing the image
    resized_im = image.resize((hight, width))
    return resized_im

def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped

import glob #globbing utility.
import numpy as np
f = open("age.csv", "a")
path = 'C:\\Users\\rahmanar\\scikit_learn_data\\lfw_home\\*.jpg'    # selecting the path of the images
for file in glob.glob(path):
    a= cv2.imread(file)
    detected_faces = detect_faces(a)
    for n, face_rect in enumerate(detected_faces):
        fa = Image.fromarray(a).crop(face_rect)
        s = np.array(resize_im(fa, 100,100))
        cv2.namedWindow('path', cv2.WINDOW_NORMAL)
        cv2.imshow('path', s)
        c = cv2.cvtColor(s, cv2.COLOR_BGR2GRAY)
        re_data = reshape(c)
        np.savetxt(f, re_data, delimiter=",", fmt = '%d')
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
# #############################################################################
# Reading the test data from the folder or server and processed

data = pd.read_csv('storage.csv', header = None)
df_x = data.iloc[:,0:].values.reshape(len(data),100,100,1)
#data.close()
f_x = np.array(df_x)

# #############################################################################
# evaluate loaded model on test data

loaded_model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])

# #############################################################################
# Predicting gender and reporting the precision, recall F1-score and accuracy

from sklearn.metrics import classification_report, confusion_matrix

y_pred = loaded_model.predict_classes(f_x)

prob = np.asmatrix(loaded_model.predict(f_x))
g_label = ["Female", "Male"]
def result(d):
    if d[0,0] > d[0,1]:
        return d[0,0], g_label[0]
    if d[0,0] < d[0,1]:
        return d[0,1], g_label[1]
gh = result(prob)
print("Confidence:", gh[0] , "Gender:", gh[1] )


In [ ]:
# #############################################################################
# 
#Full model together
#
# #############################################################################


In [3]:
# #############################################################################
# Importing necessary libraries for creating model

from keras.models import model_from_json
from keras.models import load_model
import os
import pandas as pd
import numpy as np
import keras
import cv2 # importing open CV library
import face_recognition # importing face recognition library
import imutils # Package of convenience functions
import dlib   # For frontal face identity
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
import csv
import requests

# #############################################################################
# Reading the trained model from the folder or server

json_file = open('model.json', 'r')     # Loading the jason file
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")    # Loading the weights


def detect_faces(image):                    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]  # To identify the frame size
    return face_frames


def resize_im(image, hight, width):          # Resizing the image
    resized_im = image.resize((hight, width))
    return resized_im


def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped
 
    
with open('H:\\privat\\face_female.csv') as csvfile:
    csvrows = csv.reader(csvfile, delimiter=',', quotechar='"')     # Reading websites from csv file
    for row in csvrows:
        filename = str(2)+ ".jpg"    # Putting the file name and the image formate
        url = row[1]   # Image url in the file
        result = requests.get(url, stream=True)   #Sending request to the websites
        if result.status_code == 200:
            image = result.raw.read()
            open(filename,"wb").write(image)    # Writing the images in the folder
            import glob       #globbing utility.
            import numpy as np
            f = open("storage.csv", "w")
            path = '*.jpg'    # selecting the path of the images
            for file in glob.glob(path):
                a= cv2.imread(file)
                detected_faces = detect_faces(a)   # Detecting the face or faces 
                for n, face_rect in enumerate(detected_faces):
                    fa = Image.fromarray(a).crop(face_rect)   # Cropping the face or faces
                    s = np.array(resize_im(fa, 100,100))
                    #cv2.namedWindow('path', cv2.WINDOW_NORMAL)
                    #cv2.imshow('path', s)
                    c = cv2.cvtColor(s, cv2.COLOR_BGR2GRAY)    # Converting the image
                    re_data = reshape(c)
                    np.savetxt(f, re_data, delimiter=",", fmt = '%d')  # Save the converted image data
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()    # Need to kill the image window
        
                    # #############################################################################
                    # Reading the test data from the folder or server and processed

                    data = pd.read_csv('storage.csv', header = None)
                    df_x = data.iloc[:,0:].values.reshape(len(data),100,100,1)   # Creating the requird formate
                    #data.close()
                    f_x = np.array(df_x)  # Need to convert as a array

                    # #############################################################################
                    # evaluate loaded model on test data

                    loaded_model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])

                    # #############################################################################
                    # Predicting gender and reporting the precision, recall F1-score and accuracy

                    from sklearn.metrics import classification_report, confusion_matrix

                    y_pred = loaded_model.predict_classes(f_x)    # Predicting the gender

                    prob = np.asmatrix(loaded_model.predict(f_x))   # Calculating the probability
                    g_label = ["Female", "Male"]
                    def result(d):           # Finding the requird results
                        if d[0,0] > d[0,1]:
                            return d[0,0], g_label[0]
                        if d[0,0] < d[0,1]:
                            return d[0,1], g_label[1]
                    gh = result(prob)
                    files = np.array((row[0], url, gh[0], gh[1]), dtype = str)    # Must be string data
                    d = open("gesis_female.csv", "a")
                    re_dat = reshape(files)     # Need to reshape again
                    np.savetxt(d, re_dat, delimiter=",", fmt='%s')    # Saving the output
                    print("Confidence:", gh[0] , "Gender:", gh[1] )
        print(url)
end = time.time()
print("Calculation Time:", (end - start), "seconds")       

https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcTDB56e20WiXUopZfMlkLa-4PtotqybZgxmw1JDALihM9lXrBDWqQxa1Ms
Confidence: 0.9520641 Gender: Female
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSvbtewV_D_eF5mNUYTyL9ZwOh8Sj6SSNf_KWtosHTpQQXETSCMiXEWt3o
Confidence: 0.9952075 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcS0BSJH8KYhTt5Io7FgKzD4iYb4du_54zAXILcEtKGJyUvjUsKmJs5W23Gw
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQhn7995pVdUdA_o2dIbihttVjnubG1vktR30W1UbpvgXowBfbMDiIM82AQ
Confidence: 0.62076175 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcTIn0H1H9vfDh1xOx507JNmGKkgL_-SUf7QxdqouqOclR1GpWgy1VJXBf4
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcT-1x89KCwyUidHiQPPyUVpsV4hYbWjO6jVwyMOp0lGXiBJy-NacgzPylEb
Confidence: 0.9440523 Gender: Male
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcTqPdiB5Mro5i19dZpAb2ZpI_VX7yjeTD31L3ervM1aQV-A_xiWVti2gQ
Confidence: 0.99625564 Gender: Male
https://encrypted-tbn3.gstatic.com/

https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQEEwfaMQbO2JK3E9PWKFczcJ69Te_AC3-HWthDEeSSR5cjNgzZeeP3Kg
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcTMaZcIA4WTNk6y1OwOlAxINYOwKHjMvJfmrhYH6xXma2GxBOIQZsN12ds
Confidence: 0.9658274 Gender: Male
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT18i7rPVsJwe6aA9lALe0kfB8jqUY6go_DOxBo1VrxswdOiRK1l4NfTA
Confidence: 0.99377763 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcTDTM2CumtlYlL2uVsw61N2OLol31aNN_7KVuG66lNw0y_EV2qtxnMSyDw
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS5FWeRu2iHknC-t6Og96opSLGI5td7kTv2ZfZIxnJGkcNjFWJ5I9CA-hs
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcR2rpEV38U6tXjX1iHFp_7bv7OaFpubiDFO7tBZC6XicFaB9ghZnYcUFDg
Confidence: 0.8864368 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSf2YsqqUtd0gJ2QLQKO9IZYAJstGhjnDTgoxHKY6nh4W4ugUEoWzupris
Confidence: 0.9945555 Gender: Male
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcS04Yn2K4Cwg6v9FsNb10Opr

Confidence: 0.86746514 Gender: Female
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSP2kg_jtAZ1yGpiq6PheLbYNIAyttpfRnQ99t2La-ge_y2b6GBSN5WMy0
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQE_uMGGs5LPLSxDlbF77RIx-W7rTV6A-di_mlI-b8hoDBlqllHDJ7oFvMjQA
Confidence: 0.63870645 Gender: Female
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQe3xFatu1_emAIqUmSJbFZQd_Cckh-9ypr3DENcjd1ozagf5AGqUDxRSg
Confidence: 0.7161186 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSZ9g6xWn3m3JmjcDSWJqQRP_LdPJLOaXHIUvFHJmBtemEWrZN59jk6oQ
Confidence: 0.526871 Gender: Female
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSS49xWNxqTHkjXzqKvsa3O_xf1a1OAMo9Qpzg8c9fWCX_K_GtbI5z8psQ
Confidence: 0.73591644 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcR3IYwQzLJf4tabjV7WupcMEGiX701iZIQQKxQoDcBUK5dYvN5GdFg8cg
Confidence: 0.9572026 Gender: Male
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT5tuLxY4njog-kHX0gLyctiA0u_z_NjkkrI2VrTBYEK68PtxPdunTgD

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQb3DPPNBKaMyvX3iVR1DlMiw_Mj_x2WON4pnuWsIV91ZpOwA4Mnv273w
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSjId3sNexXCDTzIubKPwF2aZ2nFRQNELNRcC0RKxzawqMcHZXWGsXqzL0
Confidence: 0.8187034 Gender: Female
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcT2WxPsprwmoS58XIyPW-r8r4w9cpP74F-rr_ffZ6GRlIQ5rMDe7C67MxG5
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREUxjln6B7M0DejCIDBLfbjE2KQ5Ez7g_E3XW3mVOYjyrjoNDNeFlyGp4
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQll2zl_LcwdwwZlKLBL7Gu711Ti2uuVwauPK_SGUtM7sCwz62AK3YSGA
Confidence: 0.85690296 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSRT1iMopS_X_x_vkev13FM5GHJ6bq_xff7q_GpMQPT70myg7sF6caQbEI
Confidence: 0.64466727 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQRN7OVd3-NsaUedctckC38_GW6LebgG2D4Vm1iBDysOwWyClZpDbvonW8
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSmg45vJxlD7e0MeAzWGqxGzifwcbL94-92de8HBPeZS46ucdJQd0EL

Confidence: 0.99607724 Gender: Male
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQH58JEErGk3J_SKkp7WRsEKKbTwPytMiW6LT3bm2rxWgmX5H2TuP0nKA
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcRJwDYNVB4ovLGJZwReyAbIkotr85a23vAopnsJPL2sZBGkb-avOrn8eA
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcTTLGYvEXoXosWp3z8PVbdyoBnTm_BNGT4OKNWL5bJQTcaSW74agPiz7w
Confidence: 0.616047 Gender: Female
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSpigqJRWdBC8VUaI1fAbyzbduPgJMws7Mbf7XoC8aJO1UL-mF0Y6uTYA56
Confidence: 0.9812448 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcS881fmyqQCm0vK3_6pIozmwmHXoaITBfGP8bk8I0b2AfoMOvuuzTSJdaY
Confidence: 0.9812448 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcS881fmyqQCm0vK3_6pIozmwmHXoaITBfGP8bk8I0b2AfoMOvuuzTSJdaY
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcRu0nyVKDfeqULQZjRxBiONfwbLlVYJjP_0Rfun0_tvhOUyv8Nv9O6BSE8
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSEaheB5Rt-_IidTn0m5N3D

...

Confidence: 0.98955566 Gender: Female
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQ1FPJ2pZohFagvLz7RlwnA6QesUUpJUWHsf1Fk0QAI14O8DnsBxHMAC-s
Confidence: 0.98955566 Gender: Female
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQ1FPJ2pZohFagvLz7RlwnA6QesUUpJUWHsf1Fk0QAI14O8DnsBxHMAC-s
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRXGrMMbWTmMuFiI6bqiONqbxBYenGVArk7JM7wuxlwC0EScB41DqKBK40
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRXGrMMbWTmMuFiI6bqiONqbxBYenGVArk7JM7wuxlwC0EScB41DqKBK40
Confidence: 0.8739741 Gender: Male
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcT0OkSrb7pe2FmivtAeVunoCf3wz2iVdmdurKp9uB_1ANcIboxUXqGY5IU
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQ5WS0YMrZ_ogkB8X5WPkhWSQHn3WmqKl6M85ZUvz_neJ6Uok5-ps_-3j4
Confidence: 0.7215513 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSApOB3OlkdwZntwlUx5MTdZQRcIcpnO3W1-Dr5aQnBaq6lVNRWvBIwWw0
Confidence: 0.7270679 Gender: Male
https://encrypted-tbn2.gstatic.com/

Confidence: 0.9966503 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcTfnkPeO-sNsv95gXY1Tv0WK0R609kDM0WS2IpcGSN3I5vtg9Ndx7pbMaY
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcR7wiTQu_MQ2-NNSzzD0E_joXt33Pb_frmr5I5tlg9sVRuRsRmdPgpWBy8
Confidence: 0.7584989 Gender: Female
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSz2NB5TMIdYlplgCshBMuMwyV532TNSgIm7DNHuug2X_-08rBH2NDdwA
Confidence: 0.7140741 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSvx3u4KbozH9-rCEMZDFcs2fCg10__vPPS4BIAReJ1fmU9WmCykTslMOEr
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcT4AceE-cUM9CRFTTvQ4aQmoiEASrS2vNhzritvEI3Lk93S5IKg1-LFoAJF
Confidence: 0.58847684 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcTBqymaVf4lC8znXeK9_dTFruc-BeaaWbo-1odD4ncviOPYeeoV0hFmEaV4mw
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQ4T4ROplWOabxiu6UpC3FM_p2lIscDgW96QUuS0VSBWCpuxGsd8pPehA
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSM2bIKRrmsxgJEK3

https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSI7AkoWHqjysziJlBXvTpX17ic9dWvUKDtr5NYmgG9_D9yB9oNEyYHPYE
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSI7AkoWHqjysziJlBXvTpX17ic9dWvUKDtr5NYmgG9_D9yB9oNEyYHPYE
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSI7AkoWHqjysziJlBXvTpX17ic9dWvUKDtr5NYmgG9_D9yB9oNEyYHPYE
Confidence: 0.8632709 Gender: Female
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcTLYcEhhLnoaFEMBBwfhM2orKnvIZeEX40Kb6l1ikc3NhaLgucrT376pH8
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFAQjfIu-i-N3R2YRTNNf8L0a9BPSJpRYIHDS-qkcCev6B0T1FWY21eus
Confidence: 0.9927006 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcTvPboOubbSf3LUPtKFhVnuNktI0jXqiFELOxiuzKGlm3pLkaLZ96F1Yhw
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQmCIYpPJjGFovxvdmJ1Fqf8YpVn3Mehl8oObmeKIYbFg0Rgb9EUdUE1GjE
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQ1k89HNA7IN3KwdkqfViXH3jVHdxReX3e4gVcegw5FI_lOa87lw9sA_EE
https://encrypted-tbn0.gstatic.

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTUI-acDCjxAJ8pv-qVPSKXaYDeXT98j1TwCTBg4TGIBbnLdyQslI2ZRAI
Confidence: 0.94226843 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcRPi1sgShMgNSYK-kAw3FBWTDMXS9q09AfzZJXDarg4VuWwFSHRX-2dBcTV
Confidence: 0.67640704 Gender: Male
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTeKWKDeGieN_ZT5ma2F6vOrXdA0El7IxlznXtWRWQ8WUEVCJ7Wl4yLBA
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQTs0Tm-HV6TfRQlmtxLhmX7UtX5CN3UUGzVv_bzRjcPV9QlCBh04rjOA
Confidence: 0.522761 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSPEmWS--kZgtTosfEJ9WBRPyOE5Jxpr83hXToFI1af1ldOwPQhVEYx1yc
Confidence: 0.67176574 Gender: Male
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQTiZGqMurVXIcXiUxZpi2jDOYcBNY3j-VB9CLsTPOSSBd2XlqZPtToUM0
Confidence: 0.9509112 Gender: Male
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcR2nj2Sv1ton-v53TPcqrmaks4VGFYkg_mRDVHB8A1_0HShrxKE-nA0IR8
https://encrypted-tbn1.gstatic.com/image

https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcS1zDN_47Wmwxt_A9mOX-kYfIIL0jeGPmQaakVDuwZ_Mn5zI1aTwMCCM2Eq
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQ8g0ep310CUnvp7exX-9wTCgioIcr5L8Un5oySav8P3IiMLB2qI8XNlU4
Confidence: 0.9946067 Gender: Female
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFh1MGMLjyaUCZF8WDX67NhMLJZFDJzPvhDsx6nfltLPRc9DeOI2kn8Rg
Confidence: 0.5346072 Gender: Male
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcSODU3-In2kI5pVg-GRkdWsxmC5GPe7IhLMhGZwCu_96XZz2u-m4kE-tw
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSo-Y1pHp-GfNYUjRNxsscnrK6xwsWY7DfRaNUBifwfoy9M7cn1w8CIUZk
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQhKf4aLxinbwsYoy0ebI-JpvJ4KANZFtc6NgmLqnAtomWFdixy_ZPEc6E
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSxPQNuHJxP92X73m-XT1cw5UKtjgFJaYdGzDaFVKpDjSq38j0D2e1-gM0
https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSra89YxmWB1BePx8wyXF3BUS1au91HhGs-K-O6IalPJzKK8d2vVrk5cac
Confidence: 0.8082969 Gender: Fe